In [1]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick
from IPython.display import display, Markdown

#import arcgis libraries
from arcgis.gis import *
gis = GIS()

In [2]:
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
dirWork      = os.getcwd()
dirData      = os.path.join(dirWork, r'data')
dirTDM       = os.path.join(dirData, r'tdm\1_WF')
dirResults   = os.path.join(dirWork, r'results')
dirLayers    = os.path.join(dirResults, r'layers')
dirProcessed = os.path.join(dirWork, r'data-processed\tdm')

In [4]:
dDirections = {
     'Dir'     : ['D1'   ,'D2'   ],
     'DirDesc' : ['NB/EB','SB/WB']
     
}
df_Directions = pd.DataFrame(dDirections)


In [5]:
shp_Forecasts = os.path.join(dirData,  r'forecasts\WFRCForecasts_2019RTP_v831_20200508_StateWideMerged_20200608h.shp')
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)

In [6]:
df_Scenarios       = pd.read_csv(os.path.join(dirResults, r'scenarios.csv'))
df_VCGroup_Seasons = pd.read_csv(os.path.join(dirResults, r'vcgroups_seasons.csv'))
df_VCGroup_DOWPeak = pd.read_csv(os.path.join(dirResults, r'vcgroups_dowpeak.csv'))
df_VCGroups        = pd.read_csv(os.path.join(dirResults, r'vcgroups.csv'))

df_VCGroups_15  = df_VCGroups[['VCGroupCode']]
df_VCGroups_Hr  = df_VCGroups[['VCGroupCode']]
df_VCGroups_Prd = df_VCGroups[df_VCGroups['HrPctOf'] == 'Prd'].copy()
df_VCGroups_Prd = df_VCGroups_Prd[['VCGroupCode']]
df_VCGroups_15 ['VCGroupCode'] = df_VCGroups_15['VCGroupCode'] + "15"
df_VCGroups_Hr ['VCGroupCode'] = df_VCGroups_Hr['VCGroupCode'] + "Hr"
df_VCGroups_Prd['VCGroupCode'] = df_VCGroups_Prd['VCGroupCode'] + "Prd"
df_VCGroups_Expanded = pd.concat([df_VCGroups_15,df_VCGroups_Hr,df_VCGroups_Prd])
    
display(df_Scenarios)
display(df_VCGroup_Seasons)
display(df_VCGroup_DOWPeak)
display(df_VCGroups_Expanded)


C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Program Files\ArcGIS\Pro\bin\Python\envs\arcgispro-py3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryCSV,PrvScenarioCode,AdjScenarioCode
0,Yes,Base19,Base,2019 Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed.csv,NaN,NaN
1,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed.csv,Base19,NaN
2,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed.csv,TIP24,NaN
3,Yes,RTP40,RTP,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed.csv,RTP30,NaN
4,Yes,RTP50,RTP,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed.csv,RTP40,NaN
5,Yes,NB30,No Build,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed.csv,TIP24,RTP30
6,Yes,NB40,No Build,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed.csv,NB30,RTP40
7,Yes,NB50,No Build,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed.csv,NB40,RTP50
8,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed.csv,TIP24,RTP30
9,Yes,UF40,Needs,2040 Needs,2040,v831_SE40_Net40UF_Summary_SEGID_Detailed.csv,UF30,RTP40


,SeasonCode,SeasonGroup,SeasonName,SeasonType
0,Ann,S00-Ann,Annual,Season
1,Win,S01-Win,Winter,Season
2,Spr,S02-Spr,Spring,Season
3,Sum,S03-Sum,Summer,Season
4,Fal,S04-Fal,Fall,Season
5,Jan,M01-Jan,January,Month
6,Feb,M02-Feb,February,Month
7,Mar,M03-Mar,March,Month
8,Apr,M04-Apr,April,Month
9,May,M05-May,May,Month


,DOWGroup,DOWPkCode,DOWPkName,HrPctOf,Prd
0,W1-Weekday,WkAM,AM - Weekday,Prd,AM
1,W1-Weekday,WkMD,Midday - Weekday,Prd,MD
2,W1-Weekday,WkPM,PM - Weekday,Prd,PM
3,5-Friday,FrPM,PM - Friday,DY,PM
4,6-Saturday,SaDY,Saturday,DY,DY
5,7-Sunday,SuDY,Sunday,DY,DY


,VCGroupCode
0,AnnWkAM15
1,WinWkAM15
2,SprWkAM15
3,SumWkAM15
4,FalWkAM15
...,...
46,AugWkPMPrd
47,SepWkPMPrd
48,OctWkPMPrd
49,NovWkPMPrd


In [7]:
df_VCGroups_Prd

,VCGroupCode
0,AnnWkAMPrd
1,WinWkAMPrd
2,SprWkAMPrd
3,SumWkAMPrd
4,FalWkAMPrd
5,JanWkAMPrd
6,FebWkAMPrd
7,MarWkAMPrd
8,AprWkAMPrd
9,MayWkAMPrd


In [8]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']
def ScenarioNameFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioName']


def DOWPkNameFromCode (sCode):
    return df_VCGroup_DOWPeak[df_VCGroup_DOWPeak['DOWPkCode'] == sCode].iloc[0]['DOWPkName']
def SeasonNameFromCode (sCode):
    return df_VCGroup_Seasons[df_VCGroup_Seasons['SeasonCode'] == sCode].iloc[0]['SeasonName']



#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

In [9]:
sCode = 'AvgWkPM15'
sCode = 'WkPM'
#DOWPkNameFromCode ()
#sCode
df_VCGroup_DOWPeak

,DOWGroup,DOWPkCode,DOWPkName,HrPctOf,Prd
0,W1-Weekday,WkAM,AM - Weekday,Prd,AM
1,W1-Weekday,WkMD,Midday - Weekday,Prd,MD
2,W1-Weekday,WkPM,PM - Weekday,Prd,PM
3,5-Friday,FrPM,PM - Friday,DY,PM
4,6-Saturday,SaDY,Saturday,DY,DY
5,7-Sunday,SuDY,Sunday,DY,DY


In [10]:
dfs_VC_MaxDirFG_wVCGroups =[]

display ('Scenario Codes:')

for (idx, row) in df_Scenarios.iterrows():    

    print(row.loc['ScenarioCode'])
    
    dfs_VC_MaxDirFG_wVCGroups.append(pd.read_csv(os.path.join(dirResults, r'temp\3_VC_MaxDirFG_wVCGroups_' + row.loc['ScenarioCode'] + '.csv')))


'Scenario Codes:'

Base19
TIP24
RTP30
RTP40
RTP50
NB30
NB40
NB50
UF30
UF40
UF50


In [11]:
dfs_VC_MaxDirFG_wVCGroups[0]

,SEGID,FT,AnnFrPM15,AnnSaDY15,AnnSuDY15,AnnWkAM15,AnnWkMD15,AnnWkPM15,AprFrPM15,AprSaDY15,...,SepWkPMPrd,SprWkAMPrd,SprWkMDPrd,SprWkPMPrd,SumWkAMPrd,SumWkMDPrd,SumWkPMPrd,WinWkAMPrd,WinWkMDPrd,WinWkPMPrd
0,0006_146.9,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,0006_149.9,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,0006_150.6,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,0006_152.6,3.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,0006_152.9,2.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3613,WFRC_8208,5.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3614,WFRC_8209,4.0,0.04,0.03,0.02,0.03,0.03,0.04,0.04,0.03,...,0.04,0.02,0.02,0.03,0.03,0.02,0.03,0.02,0.02,0.03
3615,WFRC_8213,5.0,0.06,0.05,0.03,0.03,0.04,0.05,0.07,0.05,...,0.05,0.02,0.03,0.05,0.02,0.03,0.05,0.02,0.03,0.04
3616,WFRC_8215,4.0,0.18,0.12,0.09,0.13,0.11,0.17,0.18,0.12,...,0.15,0.10,0.09,0.14,0.10,0.09,0.14,0.09,0.08,0.13


In [35]:
sScenario = 'RTP50'
sVCGroup  = 'AnnWkPMPrd'

#expression to classify enrollment
arcade_expression = ("var v = $feature." + sVCGroup + ";"
                     "var ft = $feature.FT;"
                     "if      (v<0.7  && ft<10) { return 'class_a1'; }"
                     "else if (v<0.85 && ft<10) { return 'class_a2'; }"
                     "else if (v<1.0  && ft<10) { return 'class_a3'; }"
                     "else if (v<1.5  && ft<10) { return 'class_a4'; }"
                     "else if (v>=1.5 && ft<10) { return 'class_a5'; }"
                     "else if (v<0.7  && ft>10) { return 'class_f1'; }"
                     "else if (v<0.85 && ft>10) { return 'class_f2'; }"
                     "else if (v<1.0  && ft>10) { return 'class_f3'; }"
                     "else if (v<1.5  && ft>10) { return 'class_f4'; }"
                     "else if (v>=1.5 && ft>10) { return 'class_f5'; }"
                    )

#symbology for enrollment classes
uv = [{"value":"class_a1", "label":"Arterial Less than 0.70", "symbol":{"type":"esriSLS","color":[  0,255,  0,160], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a2", "label":"Arterial 0.70 to 0.85"  , "symbol":{"type":"esriSLS","color":[169,243,106], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a3", "label":"Arterial 0.85 to 1.00"  , "symbol":{"type":"esriSLS","color":[255,228,105], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a4", "label":"Arterial 1.00 to 1.50"  , "symbol":{"type":"esriSLS","color":[255,  0,  0], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_a5", "label":"Arterial More than 1.50", "symbol":{"type":"esriSLS","color":[115,  0,  0], "width":1.5,"style":"esriSLSSolid"}},
      {"value":"class_f1", "label":"Freeway Less than 0.70" , "symbol":{"type":"esriSLS","color":[  0,255,  0,160], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f2", "label":"Freeway 0.70 to 0.85"   , "symbol":{"type":"esriSLS","color":[169,243,106], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f3", "label":"Freeway 0.85 to 1.00"   , "symbol":{"type":"esriSLS","color":[255,228,105], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f4", "label":"Freeway 1.00 to 1.50"   , "symbol":{"type":"esriSLS","color":[255,  0,  0], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class_f5", "label":"Freeway More than 1.50" , "symbol":{"type":"esriSLS","color":[115,  0,  0], "width":4.0,"style":"esriSLSSolid"}}]

idDisplay = idFromCode(sScenario)
sTitle = "Congestion: " + ScenarioNameFromCode(sScenario) + ' - ' + SeasonNameFromCode(sVCGroup[0:3]) + ' ' + DOWPkNameFromCode(sVCGroup[3:7])

sdf_Forecasts_wVC = pd.DataFrame.merge(sdf_Forecasts,dfs_VC_MaxDirFG_wVCGroups[idDisplay],on='SEGID')

sdf_Forecasts_wVC = sdf_Forecasts_wVC[['SEGID',sVCGroup,'FT','SHAPE']]

#create map centered on Salt Lake
map_seg = gis.map('Salt Lake')

#define sdf layer
sdf_Forecasts_wVC.spatial.plot(map_widget = map_seg,renderer_type='u-a',unique_values=uv,arcade_expression=arcade_expression,default_symbol="",arcade_title="V/C Ratio Range")

#define map characteristics
map_seg.layout.height='800px'
map_seg.legend=True

#map title
display(Markdown('<h2><center>' + sTitle + '</center></h2>'))

#display map
map_seg

<h2><center>Congestion: 2050 RTP - Annual PM - Weekday</center></h2>

MapView(layout=Layout(height='800px', width='100%'), legend=True)

In [36]:
sdf_Forecasts_wVC

,SEGID,AnnWkPMPrd,FT,SHAPE
0,0068_036.0,0.79,2.0,"{""paths"": [[[421160.5499999998, 4476386.34], [..."
1,WFRC_8000,1.12,2.0,"{""paths"": [[[420067.91000000015, 4479534.80000..."
2,0131_001.7,0.67,2.0,"{""paths"": [[[422222.28000000026, 4480359.77], ..."
3,0068_038.1,1.16,2.0,"{""paths"": [[[420067.91000000015, 4479534.80000..."
4,2046_002.3,0.86,4.0,"{""paths"": [[[422575.00519999955, 4478386.86700..."
...,...,...,...,...
3890,MAG_6247,0.49,4.0,"{""paths"": [[[437218.06879999954, 4461802.4725]..."
3891,MAG_6248,1.95,33.0,"{""paths"": [[[435487.7599999998, 4462445.02], [..."
3892,MAG_6249,0.41,2.0,"{""paths"": [[[442185.8794999998, 4440527.9736],..."
3893,MAG_6250,0.67,2.0,"{""paths"": [[[442531.98900000006, 4440253.285],..."


In [37]:
df_VC = pd.read_csv(os.path.join(dirResults, r'1_VC_' + sScenario + '.csv'))

In [38]:
sScenario

'RTP50'

In [39]:
df_VC_filtered = df_VC[(df_VC['SEGID']=='2034_000.0') & (df_VC['VCGroupCode']==sVCGroup[0:7])]
pd.set_option('display.max_columns', None)
df_VC_filtered

,SEGID,AREATYPE,SegDYVol_TDM,FUNCGROUP,FGFAC,Dir,LANES,FT,FTCLASS,CAP1HL,DFAC,Prd_TDM,PRDFAC,MDPERCENT,HVPERCENT,FYEAR,ForecastAADT,ForecastAADTAdj,AvgAnnualPrdVol,DOWFACFC,SsnGrp,SsnATGroup,SsnVolCls,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,AvgSsnPrdVol,SiteGroupDOW,DOWGroup,DOWFactor,AvgSsnDOWPrdVol,Prd,HrPctOf,VolPkHrPct,SeasonType,VCGroupCode,PrdHrs,AvgSsnDOWPkHrVol,TrkFac,PrdPCEFlow,PkHrPCEFlow,PrdVC,PkHrVC,PHF,Pk15PCEFlow,Pk15VC
411548,2034_000.0,3,2446.8,Arterial,1.0,D1,1,5,Collector,525,0.5373,PM,0.2482,8.4,4.5,2050,15000,15000,2000.3679,Arterial,StatewideArterial,3-Suburban,0-AllVolumes,0,1000000,SNT,S00-Ann,1.0,2000,WD2,W1-Weekday,1.12,2235,PM,Prd,0.3721,Season,AnnWkPM,3,831.6435,1.0393,774,864,1.47,1.65,0.98,881,1.68
411549,2034_000.0,3,2446.8,Arterial,1.0,D2,1,5,Collector,525,0.4627,PM,0.2482,11.0,5.6,2050,15000,15000,1722.6321,Arterial,StatewideArterial,3-Suburban,0-AllVolumes,0,1000000,SNT,S00-Ann,1.0,1722,WD2,W1-Weekday,1.12,1925,PM,Prd,0.3721,Season,AnnWkPM,3,716.2925,1.0500,673,752,1.28,1.43,0.98,767,1.46


In [36]:
try:
    df_ForecastAdj = pd.read_csv(os.path.join(dirResults, r'ForecastAdj_' + sScenario + '.csv'))
except Exception:
    display ('No forecast adjustment')
    pass

'No forecast adjustment'

In [37]:
try:
    df_ForecastAdj_filtered = df_ForecastAdj[(df_ForecastAdj['SEGID']=='0085_011.3')]
    df_ForecastAdj_filtered
except Exception:
    display ('No forecast adjustment')
    pass  

'No forecast adjustment'

# Create Layers

In [38]:
for (idx, row) in df_Scenarios.iterrows():

    sScenario = row.loc['ScenarioCode']

    idLoc = idFromCode(sScenario)
    display("Congestion: " + ScenarioNameFromCode(sScenario) + ' - ' + SeasonNameFromCode(sVCGroup[0:3]) + ' ' + DOWPkNameFromCode(sVCGroup[3:7]))

    sdf_Forecasts_wVC = pd.DataFrame.merge(sdf_Forecasts,dfs_VC_MaxDirFG_wVCGroups[idLoc],on='SEGID')

    sdf_Forecasts_wVC['Scenario'] = sScenario
    
    outFields = ['Scenario','SEGID','FT','SHAPE'] + list(df_VCGroups_Expanded.VCGroupCode.unique())
    
    sdf_Forecasts_wVC = sdf_Forecasts_wVC[outFields]
    
    sdf_Forecasts_wVC = sdf_Forecasts_wVC.fillna(0)
    #display(sdf_Forecasts_wVC)

    try:
        sdf_Forecasts_wVC.spatial.to_featureclass(location=os.path.join(dirLayers,'VC_' + sScenario + '.shp'))
    except Exception:
        display ('FAIL')
        pass
    
    



'Congestion: 2019 Base - Annual PM - Weekday'

'Congestion: 2024 TIP - Annual PM - Weekday'

'Congestion: 2030 RTP - Annual PM - Weekday'

'Congestion: 2040 RTP - Annual PM - Weekday'

'Congestion: 2050 RTP - Annual PM - Weekday'

...

'Congestion: 2040 No Build - Annual PM - Weekday'

'Congestion: 2050 No Build - Annual PM - Weekday'

'Congestion: 2030 Needs - Annual PM - Weekday'

'Congestion: 2040 Needs - Annual PM - Weekday'

'Congestion: 2050 Needs - Annual PM - Weekday'

# Segment Data for WebApp

In [42]:
dfs_VC = []

for (idx, row) in df_Scenarios.iterrows():

    df_VC = pd.read_csv(os.path.join(dirResults, r'1_VC_' + row.loc['ScenarioCode'] + '.csv'))
    
    dfs_VC.append(df_VC)
    

In [43]:
dReportingColumns_Seg           = ['SEGID'     ,'ForecastAADT'     ,'ForecastAADTAdj'       ]
dReportingColumns_Seg_Names     = ['Segment ID','RTP Forecast AADT','Scenario Forecast AADT']
dReportingColumns_Seg_jsonNames = ['SegID','Forecast','ForecastAdj']

dReportingColumns_Data           = ['VCGroupCode'  ,'FUNCGROUP'       ,'FGFAC'                  ,'SiteGroupSeason'  ,'SeasonFactor' ,'DOWFactor'         ,'PRDFAC'       ,'VolPkHrPct',     'Dir'      ,'DFAC'            ,'MDPERCENT'     ,'HVPERCENT'    ,'TrkFac'      ,'PHF','LANES','AREATYPE' ,'FT'             ,'Capacity'      ,'PrdPCEFlow'       ,'PkHrPCEFlow'         ,'Pk15PCEFlow'              ,'PrdVC'     ,'PkHrVC'       ,'Pk15VC'         ]
dReportingColumns_Data_Names     = ['VC Group Code','Functional Group','Functional Group Factor','Season Group Code','Season Factor','Day-of-Week Factor','Period Factor','Max Hour Factor','Direction','Direction Factor','Medium Truck %','Heavy Truck %','Truck Factor','PHF','Lanes','Area Type','Functional Type','Capacity (vph)','Period Flow (vph)','Peak Hour Flow (vph)','Peak 15-Minute Flow (vph)','Period V/C','Peak Hour V/C','Peak 15-Min V/C']
dReportingColumns_Data_jsonNames = ['VCG'          ,'FG'              ,'FGF'                    ,'SG'               ,'SF'           ,'DOW'               ,'PF'           ,'HF'             ,'D'        ,'DF'              ,'MD'            ,'HV'           ,'TF'          ,'PHF','LN'   ,'AT'       ,'FT'             ,'CP'            ,'PrdF'             ,'PkF'                 ,'15F'                      ,'PrdVC'     ,'PkVC'         ,'15VC'           ]

dReportingColumns_All = dReportingColumns_Seg + dReportingColumns_Data
dReportingColumns_All_jsonNames = dReportingColumns_Seg_jsonNames + dReportingColumns_Data_jsonNames
dReportingColumns_All_jsonNames


['SegID',
 'Forecast',
 'ForecastAdj',
 'VCG',
 'FG',
 'FGF',
 'SG',
 'SF',
 'DOW',
 'PF',
 'HF',
 'D',
 'DF',
 'MD',
 'HV',
 'TF',
 'PHF',
 'LN',
 'AT',
 'FT',
 'CP',
 'PrdF',
 'PkF',
 '15F',
 'PrdVC',
 'PkVC',
 '15VC']

In [44]:

for index, df_VC in enumerate(dfs_VC):

    sScenarioCode = df_Scenarios['ScenarioCode'][index]
    display (sScenarioCode)
    
    #dfOutputSegs = df_VC.groupby(dReportingColumns_Seg, as_index=False).size().reset_index(name='Freq')
    #dfOutputSegs = dfOutputSegs[dReportingColumns_Seg]
    
    df_VC['ForecastAADT']    = df_VC.apply(lambda x: "{:,.0f}".format(x['ForecastAADT']), axis=1)
    df_VC['ForecastAADTAdj'] = df_VC.apply(lambda x: "{:,.0f}".format(x['ForecastAADTAdj']), axis=1)
    
    #dfOutputSegs.columns  = dReportingColumns_Seg_jsonNames
    #dfOutputSegs.to_json(os.path.join(dirResults, 'WebAppData\\segment_vcdata\\_scenariosegs_' + sScenarioCode + ".json"),orient='records')
    
    df_VC['Capacity']        = df_VC['CAP1HL'] * df_VC['LANES']

    df_VC['FGFAC'          ] = df_VC.apply(lambda x: "{:.3f}" .format(x['FGFAC'          ]), axis=1)    
    df_VC['PRDFAC'         ] = df_VC.apply(lambda x: "{:.3f}" .format(x['PRDFAC'         ]), axis=1)    
    df_VC['VolPkHrPct'     ] = df_VC.apply(lambda x: "{:.3f}" .format(x['VolPkHrPct'     ]), axis=1)    
    df_VC['SeasonFactor'   ] = df_VC.apply(lambda x: "{:.3f}" .format(x['SeasonFactor'   ]), axis=1)
    df_VC['DOWFactor'      ] = df_VC.apply(lambda x: "{:.3f}" .format(x['DOWFactor'      ]), axis=1)
    df_VC['DFAC'           ] = df_VC.apply(lambda x: "{:.3f}" .format(x['DFAC'           ]), axis=1)
    
    df_VC['Capacity'       ] = round((df_VC['Capacity'   ]/10),0)*10
    df_VC['PrdPCEFlow'     ] = round((df_VC['PrdPCEFlow' ]/10),0)*10
    df_VC['PkHrPCEFlow'    ] = round((df_VC['PkHrPCEFlow']/10),0)*10
    df_VC['Pk15PCEFlow'    ] = round((df_VC['Pk15PCEFlow']/10),0)*10
    
    df_VC['Capacity'       ] = df_VC.apply(lambda x: "{:,.0f}".format(x['Capacity'   ]), axis=1)
    df_VC['PrdPCEFlow'     ] = df_VC.apply(lambda x: "{:,.0f}".format(x['PrdPCEFlow' ]), axis=1)
    df_VC['PkHrPCEFlow'    ] = df_VC.apply(lambda x: "{:,.0f}".format(x['PkHrPCEFlow']), axis=1)
    df_VC['Pk15PCEFlow'    ] = df_VC.apply(lambda x: "{:,.0f}".format(x['Pk15PCEFlow']), axis=1)
    
    df_VC['MDPERCENT'      ] = df_VC.apply(lambda x: "{:.1f}".format(x['MDPERCENT']), axis=1)
    df_VC['HVPERCENT'      ] = df_VC.apply(lambda x: "{:.1f}".format(x['HVPERCENT']), axis=1)
    
    df_VC['TrkFac'         ] = df_VC.apply(lambda x: "{:.3f}".format(x['TrkFac']), axis=1)
    df_VC['PrdVC'          ] = df_VC.apply(lambda x: "{:.2f}".format(x['PrdVC' ]), axis=1)
    df_VC['PkHrVC'         ] = df_VC.apply(lambda x: "{:.2f}".format(x['PkHrVC']), axis=1)
    df_VC['Pk15VC'         ] = df_VC.apply(lambda x: "{:.2f}".format(x['Pk15VC']), axis=1)  
    
#    #fVCforSet = df_VC.groupby(['SEGID','VCGroupCode'], as_index=False).size().reset_index(name='Freq')
#    
#    for index, row in dfOutputSegs.iterrows():
#
#        sSegID = row['SegID']
#        #sVCGroupCode = row['VCGroupCode']
#          
#        #dfOutputData = df_VC[(df_VC['SEGID'] == sSegID) & (df_VC['VCGroupCode'] == sVCGroupCode)]
#        
#        dfOutputData = df_VC[(df_VC['SEGID'] == sSegID)]
#
#        #sFilename = sScenarioCode + '_' + sSegID + '_ '+ sVCGroupCode + ".json"
#        sFilename = sScenarioCode + '_' + sSegID + ".json"
#        #display(sFilename)
#        
#        dfOutputData = dfOutputData[dReportingColumns_Data]
#        
#        dfOutputData.columns  = dReportingColumns_Data_jsonNames
#        
#        dfOutputData.to_json(os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sFilename),orient='records')
    

'Base19'

'TIP24'

'RTP30'

'RTP40'

'RTP50'

...

'NB40'

'NB50'

'UF30'

'UF40'

'UF50'

In [45]:
for index, df_VC in enumerate(dfs_VC):

    sScenarioCode = df_Scenarios['ScenarioCode'][index]
    display (sScenarioCode)
    
    dfOutputSegs = df_VC.groupby(dReportingColumns_Seg, as_index=False).size().reset_index(name='Freq')
    
    for index, row in dfOutputSegs.iterrows():

        sSegID = row['SEGID']
        
        dfOutputData = df_VC[(df_VC['SEGID'] == sSegID)]
        
        dfOutputDataNew = dfOutputData[dReportingColumns_All].copy()
        dfOutputDataNew.columns = dReportingColumns_All_jsonNames
        
        sFilename = sScenarioCode + '_' + sSegID + ".json"
        
#        group = merge_hr_anr.groupby('EMPCODE').groups
#        d = {'EMPCODE': list(group.keys())[0], 'Indicators': [{'IndicatorName': merge_hr_anr.loc[i, 'Indicator']} for i in list(group.values())[0].unique()]}
#        d['Indicators'] = list(map(dict,sorted(set(map(lambda x: tuple(x.items()),d['Indicators'])), key=list(map(lambda x: tuple(x.items()),d['Indicators'])).index)))
#        d['Performance'] = [{i['IndicatorName']: merge_hr_anr.loc[merge_hr_anr['Indicator'].eq(i['IndicatorName']), 'CustNAME'].dropna().tolist()} for i in d['Indicators']]
#        print(d)
#        
        j = (dfOutputDataNew.groupby(dReportingColumns_Seg_jsonNames, as_index=False)
                .apply(lambda x: x[dReportingColumns_Data_jsonNames]
                .to_dict('r'))
                .reset_index()
                .rename(columns={0:'VCData'})
                .to_json(os.path.join(dirResults, "WebAppData\\segment_vcdata\\" + sFilename),orient='records'))#, indent=2))
        
        #
        #display(j)
        
        #break
    

'Base19'

'TIP24'

'RTP30'

'RTP40'

'RTP50'

...

'NB40'

'NB50'

'UF30'

'UF40'

'UF50'

In [ ]:
dfOutputDataNew

In [15]:
dfOutputSegs

,SegID,Forecast,ForecastAdj
0,0006_146.9,0,0
1,0006_149.9,0,0
2,0006_150.6,0,0
3,0006_152.6,0,0
4,0006_152.9,0,0
...,...,...,...
3613,WFRC_8208,0,0
3614,WFRC_8209,500,500
3615,WFRC_8213,500,500
3616,WFRC_8215,"3,700","3,700"


# Seg WebApp Data

In [ ]:
#column names from TDM
nameSegID   = 'SEGID'
nameFG      = 'FUNCGROUP'
nameLinks   = 'LINKS'
nameLanes   = 'LANES'
nameFT      = 'FT'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

nameDir     = 'Dir'

indexMelt   = [nameSegID,nameFG,'D1_' + nameFT,'D2_' + nameFT,'D1_' + nameLanes,'D2_' + nameLanes]

df_TDMcmb = pd.DataFrame() 

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Importing')

    csvFileName = os.path.join(dirProcessed, row.SegSummaryDBF + ".csv")
    #csvFileName = os.path.splitext(dbfFileName)[0]+'.csv'

    df_TDM_import = pd.read_csv(csvFileName)

    #Remove where Total FG
    df_TDM_import = df_TDM_import[df_TDM_import.FUNCGROUP != 'Total']
    #display(df_TDM_import)
        
    print(row.loc[nameS] + ' Processing.')

    #normalize data with period/volume type fields
    df_TDM_melt = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=['D1_LINKS','D2_LINKS'])
    df_TDM_melt[nameDir] = df_TDM_melt['variable'].str.get(0) + df_TDM_melt['variable'].str.get(1)
    
    df_TDM_melt = df_TDM_melt.rename(columns={"value": nameLinks})
    
    #display(df_TDM_melt)
    
    #calculate direction level links, lanes, capacity
   
    df_TDM_melt[nameLanes] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameLanes], df_TDM_melt['D2_'+nameLanes])
    df_TDM_melt[nameFT] = np.where(df_TDM_melt[nameDir].eq('D1'), df_TDM_melt['D1_'+nameFT], df_TDM_melt['D2_'+nameFT])
    
    df_TDM_melt = df_TDM_melt.drop(columns=['variable','D1_'+nameLanes,'D2_'+nameLanes,'D1_'+nameFT,'D2_'+nameFT])
    #display(df_TDM_melt)

    df_TDM_melt['ScenarioCode'] = row.loc['ScenarioCode']
    
    df_TDM_meltn = df_TDM_melt[['ScenarioCode','SEGID','FUNCGROUP','Dir','FT','LANES']]
    
    df_TDMcmb = pd.concat([df_TDMcmb,df_TDM_meltn],ignore_index=True)
    
print('Done')

In [ ]:
df_TDMcmb

In [ ]:
df_SegIDsAll = df_TDMcmb.groupby(['SEGID'], as_index=False).agg({'ScenarioCode':[np.size]})
df_SegIDsAll.columns = ['SEGID','numRecords']
df_SegIDsAll


In [11]:
import itertools

#hours within period, for capacity
dFG = {
     'FUNCGROUP'     : ['Arterial','Freeway','Managed','CD Road']
}
df_FG = pd.DataFrame(dFG)

#hours within period, for capacity
dDir = {
     'Dir' : ['D1','D2']
}
df_Dir = pd.DataFrame(dDir)


#create df with all possible season and day-of-weak/peak groups
df_SegIDsFGAll = pd.DataFrame(list(itertools.product(df_SegIDsAll['SEGID'],df_FG['FUNCGROUP'],df_Scenarios['ScenarioCode'],df_Dir['Dir'])),columns=['SEGID','FUNCGROUP','ScenarioCode','Dir'])
display(df_SegIDsFGAll)

df_ScSegFGDir = pd.DataFrame.merge(df_SegIDsFGAll,df_TDMcmb,on=('ScenarioCode','SEGID','FUNCGROUP','Dir'),how='left')
display(df_ScSegFGDir)

NameError: name 'df_SegIDsAll' is not defined

In [12]:
df_ScSegFGDir.groupby(['ScenarioCode'],as_index=False).agg({'SEGID':[np.size]})

NameError: name 'df_ScSegFGDir' is not defined

In [13]:
df_ScSegFGDir_nonbase = df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']!='Base19']
display(df_ScSegFGDir_nonbase)

NameError: name 'df_ScSegFGDir' is not defined

In [14]:
df_ScenariosToPrevious = df_Scenarios[['ScenarioCode','PrvScenarioCode']]
df_ScSegFGDir_nonbase_wpreviouscode = pd.DataFrame.merge(df_ScSegFGDir_nonbase,df_ScenariosToPrevious,on='ScenarioCode')
display(df_ScSegFGDir_nonbase_wpreviouscode)

NameError: name 'df_ScSegFGDir_nonbase' is not defined

In [15]:
df_ScSegFGDir_nonbase_wpreviouscode[df_ScSegFGDir_nonbase_wpreviouscode['PrvScenarioCode']==0]

NameError: name 'df_ScSegFGDir_nonbase_wpreviouscode' is not defined

In [16]:
df_ScSegFGDir_nonbase_wpreviousdata = pd.DataFrame.merge(df_ScSegFGDir_nonbase_wpreviouscode,df_ScSegFGDir,left_on=('SEGID','FUNCGROUP','PrvScenarioCode','Dir'),right_on=('SEGID','FUNCGROUP','ScenarioCode','Dir'),how='left')
df_ScSegFGDir_nonbase_wpreviousdata = df_ScSegFGDir_nonbase_wpreviousdata[['SEGID','ScenarioCode_x','ScenarioCode_y','FUNCGROUP','Dir','FT_x','FT_y','LANES_x','LANES_y']]
df_ScSegFGDir_nonbase_wpreviousdata = df_ScSegFGDir_nonbase_wpreviousdata.fillna(0)
display(df_ScSegFGDir_nonbase_wpreviousdata)

NameError: name 'df_ScSegFGDir_nonbase_wpreviouscode' is not defined

In [29]:
df_ScSegFGDir_nonbase_wpreviousdata[(df_ScSegFGDir_nonbase_wpreviousdata['ScenarioCode_y']==0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y


In [30]:
df_ScSegFGDir_nonbase_wpreviousdata['Change_FT'] = df_ScSegFGDir_nonbase_wpreviousdata['FT_x'      ] - df_ScSegFGDir_nonbase_wpreviousdata['FT_y'      ]
df_ScSegFGDir_nonbase_wpreviousdata['Change_LN'] = df_ScSegFGDir_nonbase_wpreviousdata['LANES_x'   ] - df_ScSegFGDir_nonbase_wpreviousdata['LANES_y'   ]
#df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']] = df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']].mask(df_VCcmb_scenario_wpreviousdata[['Change_AT','Change_FT','Change_LN']] < 0, 0)
df_ScSegFGDir_nonbase_wpreviousdata

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN
0,0006_146.9,TIP24,Base19,Arterial,D1,2.0,2.0,1.0,1.0,0.0,0.0
1,0006_146.9,TIP24,Base19,Arterial,D2,2.0,2.0,1.0,1.0,0.0,0.0
2,0006_146.9,TIP24,Base19,Freeway,D1,0.0,0.0,0.0,0.0,0.0,0.0
3,0006_146.9,TIP24,Base19,Freeway,D2,0.0,0.0,0.0,0.0,0.0,0.0
4,0006_146.9,TIP24,Base19,Managed,D1,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
314635,WFRC_8219,UF50,UF40,Freeway,D2,0.0,0.0,0.0,0.0,0.0,0.0
314636,WFRC_8219,UF50,UF40,Managed,D1,0.0,0.0,0.0,0.0,0.0,0.0
314637,WFRC_8219,UF50,UF40,Managed,D2,0.0,0.0,0.0,0.0,0.0,0.0
314638,WFRC_8219,UF50,UF40,CD Road,D1,0.0,0.0,0.0,0.0,0.0,0.0


In [31]:
#delete lines with all zeros

#df_Changes = df_ScSegFGDir_nonbase_wpreviousdata.copy()

df_Changes = df_ScSegFGDir_nonbase_wpreviousdata[(df_ScSegFGDir_nonbase_wpreviousdata['Change_FT']!=0) | (df_ScSegFGDir_nonbase_wpreviousdata['Change_LN']!=0)].copy()

#df_ScSegFGDir_nonbase_wpreviousdata

In [32]:
df_Changes['Change_LN_Text'] = df_Changes['Change_LN'].abs().astype(str)
df_Changes['Change_LN_Text'] = df_Changes['Change_LN_Text'].replace(to_replace = "\.0+$",value = "", regex = True)
df_Changes['Change_LN_Description'] = df_Changes.apply(lambda row: row['Change_LN_Text'] + " " + row['FUNCGROUP'].lower() + " lanes added" if row['Change_LN'] > 1 else "1 " + row['FUNCGROUP'].lower() + " lane added" if row['Change_LN'] == 1 else row['Change_LN_Text'] + " " + row['FUNCGROUP'].lower() + " lanes removed" if row['Change_LN'] < -1 else "1 " + row['FUNCGROUP'].lower() + " lane removed" if row['Change_LN'] == -1 else "", axis=1)

In [33]:
df_Changes[df_Changes['Change_LN']<0]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description
4272,0089_398.7,TIP24,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4273,0089_398.7,TIP24,Base19,Arterial,D2,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4280,0089_399.7,TIP24,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4281,0089_399.7,TIP24,Base19,Arterial,D2,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4288,0089_400.5,TIP24,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
...,...,...,...,...,...,...,...,...,...,...,...,...,...
312329,MAG_6136,UF50,UF40,Arterial,D2,0.0,13.0,0.0,1.0,-13.0,-1.0,1,1 arterial lane removed
312344,MAG_6138,UF50,UF40,Arterial,D1,0.0,13.0,0.0,2.0,-13.0,-2.0,2,2 arterial lanes removed
312345,MAG_6138,UF50,UF40,Arterial,D2,0.0,13.0,0.0,2.0,-13.0,-2.0,2,2 arterial lanes removed
312568,MAG_6172,UF50,UF40,Arterial,D1,0.0,13.0,0.0,2.0,-13.0,-2.0,2,2 arterial lanes removed


In [34]:
df_Changes[df_Changes['SEGID']=='0089_398.7']

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description
4272,0089_398.7,TIP24,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4273,0089_398.7,TIP24,Base19,Arterial,D2,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed
4274,0089_398.7,TIP24,Base19,Freeway,D1,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added
4275,0089_398.7,TIP24,Base19,Freeway,D2,33.0,0.0,3.0,0.0,33.0,3.0,3,3 freeway lanes added


In [35]:
df_Changes['Change_FT_Text'] = df_Changes['Change_FT'].abs().astype(str)
df_Changes['Change_FT_Description'] = df_Changes.apply(lambda row: row['FUNCGROUP'].lower() + " operational changes" if (row['Change_FT'] > 0) & (row['FT_y'] > 0) else row['FUNCGROUP'].lower() + " operational changes" if (row['Change_FT'] < 0) & (row['FT_y'] > 0) else "", axis=1)

In [36]:
df_Changes[(df_Changes['Change_FT']<0) & (df_Changes['FT_y']>0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description
512,0015_282.2,TIP24,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes
513,0015_282.2,TIP24,Base19,Arterial,D2,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes
520,0015_283.3,TIP24,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes
521,0015_283.3,TIP24,Base19,Arterial,D2,3.0,4.0,3.0,1.0,-1.0,2.0,2,2 arterial lanes added,1.0,arterial operational changes
4272,0089_398.7,TIP24,Base19,Arterial,D1,4.0,14.0,1.0,2.0,-10.0,-1.0,1,1 arterial lane removed,10.0,arterial operational changes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
312936,MAG_6231,UF50,UF40,Arterial,D1,2.0,6.0,1.0,1.0,-4.0,0.0,0,,4.0,arterial operational changes
312937,MAG_6231,UF50,UF40,Arterial,D2,2.0,6.0,1.0,1.0,-4.0,0.0,0,,4.0,arterial operational changes
312969,MAG_6237,UF50,UF40,Arterial,D2,13.0,14.0,1.0,1.0,-1.0,0.0,0,,1.0,arterial operational changes
313368,WFRC_8057,UF50,UF40,Arterial,D1,4.0,5.0,1.0,1.0,-1.0,0.0,0,,1.0,arterial operational changes


In [54]:
df_Changes_wDirections = pd.DataFrame.merge(df_Changes,df_Directions,on='Dir',how='left')

df_Changes_wDirections['Change_Conjunction'] = df_Changes_wDirections.apply(lambda row: ' and ' if (row['Change_FT_Description'] != '') & (row['Change_LN_Description'] != '') else '', axis=1)

df_Changes_wDirections['Change_Text'] = df_Changes_wDirections['DirDesc'] + ": " + df_Changes_wDirections['Change_LN_Description'] + df_Changes_wDirections['Change_Conjunction']  + df_Changes_wDirections['Change_FT_Description'] 
df_Changes_wDirections

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description,DirDesc,Change_Conjunction,Change_Text
0,0006_159.8,TIP24,Base19,Arterial,D1,2.0,2.0,2.0,1.0,0.0,1.0,1,1 arterial lane added,0.0,,NB/EB,,NB/EB: 1 arterial lane added
1,0006_159.8,TIP24,Base19,Arterial,D2,2.0,2.0,2.0,1.0,0.0,1.0,1,1 arterial lane added,0.0,,SB/WB,,SB/WB: 1 arterial lane added
2,0015_278.5,TIP24,Base19,Freeway,D2,34.0,34.0,5.0,4.0,0.0,1.0,1,1 freeway lane added,0.0,,SB/WB,,SB/WB: 1 freeway lane added
3,0015_279.6,TIP24,Base19,Freeway,D1,33.0,33.0,5.0,3.0,0.0,2.0,2,2 freeway lanes added,0.0,,NB/EB,,NB/EB: 2 freeway lanes added
4,0015_279.6,TIP24,Base19,Freeway,D2,33.0,33.0,5.0,3.0,0.0,2.0,2,2 freeway lanes added,0.0,,SB/WB,,SB/WB: 2 freeway lanes added
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5438,WFRC_8200,UF50,UF40,Arterial,D2,3.0,0.0,2.0,0.0,3.0,2.0,2,2 arterial lanes added,3.0,,SB/WB,,SB/WB: 2 arterial lanes added
5439,WFRC_8204,UF50,UF40,Arterial,D1,4.0,0.0,1.0,0.0,4.0,1.0,1,1 arterial lane added,4.0,,NB/EB,,NB/EB: 1 arterial lane added
5440,WFRC_8204,UF50,UF40,Arterial,D2,4.0,0.0,1.0,0.0,4.0,1.0,1,1 arterial lane added,4.0,,SB/WB,,SB/WB: 1 arterial lane added
5441,WFRC_8214,UF50,UF40,Arterial,D1,14.0,0.0,1.0,0.0,14.0,1.0,1,1 arterial lane added,14.0,,NB/EB,,NB/EB: 1 arterial lane added


In [55]:
df_Changes_wDirections[(df_Changes_wDirections['Change_FT']<0) & (df_Changes_wDirections['Change_LN']>0)]

,SEGID,ScenarioCode_x,ScenarioCode_y,FUNCGROUP,Dir,FT_x,FT_y,LANES_x,LANES_y,Change_FT,Change_LN,Change_LN_Text,Change_LN_Description,Change_FT_Text,Change_FT_Description,DirDesc,Change_Conjunction,Change_Text
5,0015_282.2,TIP24,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
6,0015_282.2,TIP24,Base19,Arterial,D2,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,SB/WB,and,SB/WB: 1 arterial lane added and arterial oper...
11,0015_283.3,TIP24,Base19,Arterial,D1,3.0,4.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
12,0015_283.3,TIP24,Base19,Arterial,D2,3.0,4.0,3.0,1.0,-1.0,2.0,2,2 arterial lanes added,1.0,arterial operational changes,SB/WB,and,SB/WB: 2 arterial lanes added and arterial ope...
193,2878_005.9,TIP24,Base19,Arterial,D1,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5238,MAG_6054,UF50,UF40,Arterial,D2,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,SB/WB,and,SB/WB: 1 arterial lane added and arterial oper...
5239,MAG_6062,UF50,UF40,Arterial,D1,2.0,3.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...
5240,MAG_6062,UF50,UF40,Arterial,D2,2.0,3.0,2.0,1.0,-1.0,1.0,1,1 arterial lane added,1.0,arterial operational changes,SB/WB,and,SB/WB: 1 arterial lane added and arterial oper...
5279,MAG_6170,UF50,UF40,Arterial,D1,2.0,4.0,2.0,1.0,-2.0,1.0,1,1 arterial lane added,2.0,arterial operational changes,NB/EB,and,NB/EB: 1 arterial lane added and arterial oper...


In [58]:
df_Changes_HTML = df_Changes_wDirections.groupby(['SEGID','ScenarioCode_x'])['Change_Text'].apply('<br/>'.join).reset_index()
df_Changes_HTML = pd.DataFrame.merge(df_Changes_HTML,df_Scenarios,left_on='ScenarioCode_x',right_on='ScenarioCode',how='left')
df_Changes_HTML['Change_HTML'] = "<b>" + df_Changes_HTML['ScenarioName'] + ":</b><br/>" + df_Changes_HTML['Change_Text']
df_Changes_HTML

,SEGID,ScenarioCode_x,Change_Text,Process,ScenarioCode,ScenarioGroup,ScenarioName,ScenarioYear,SegSummaryDBF,PrvScenarioCode,AdjScenarioCode,Change_HTML
0,0006_159.6,RTP30,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,TIP24,NaN,<b>2030 RTP:</b><br/>NB/EB: 1 arterial lane ad...
1,0006_159.6,UF30,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30,<b>2030 Needs:</b><br/>NB/EB: 1 arterial lane ...
2,0006_159.8,TIP24,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,Base19,NaN,<b>2024 TIP:</b><br/>NB/EB: 1 arterial lane ad...
3,0006_160.3,RTP30,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,TIP24,NaN,<b>2030 RTP:</b><br/>NB/EB: 1 arterial lane ad...
4,0006_160.3,UF30,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,UF30,Needs,2030 Needs,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,TIP24,RTP30,<b>2030 Needs:</b><br/>NB/EB: 1 arterial lane ...
...,...,...,...,...,...,...,...,...,...,...,...,...
2470,WFRC_8214,UF50,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,UF50,Needs,2050 Needs,2050,v831_SE50_Net50UF_Summary_SEGID_Detailed_BH,UF40,RTP50,<b>2050 Needs:</b><br/>NB/EB: 1 arterial lane ...
2471,WFRC_8216,TIP24,NB/EB: 2 freeway lanes added<br/>SB/WB: 2 free...,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,Base19,NaN,<b>2024 TIP:</b><br/>NB/EB: 2 freeway lanes ad...
2472,WFRC_8217,TIP24,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,TIP24,TIP,2024 TIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,Base19,NaN,<b>2024 TIP:</b><br/>NB/EB: 1 arterial lane ad...
2473,WFRC_8219,RTP30,NB/EB: 1 arterial lane added<br/>SB/WB: 1 arte...,Yes,RTP30,RTP,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,TIP24,NaN,<b>2030 RTP:</b><br/>NB/EB: 1 arterial lane ad...


In [40]:
df_ScSegFGDir_base = df_ScSegFGDir[df_ScSegFGDir['ScenarioCode']=='Base19']
#display(df_ScSegFGDir_base)

df_ScSegFGDir_baseNoZero = df_ScSegFGDir_base[df_ScSegFGDir_base['LANES']>0]

df_FTs = pd.read_csv(os.path.join(dirTDM, r'FTs.csv'))
df_ScSegFGDir_baseNoZero = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero,df_FTs,on='FT',how='left')

df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero['LANES'].abs().astype(str)
df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero['LANES_Text'].replace(to_replace = "\.0+$",value = "", regex = True)

df_ScSegFGDir_baseNoZero['LANES_Text'] = df_ScSegFGDir_baseNoZero.apply(lambda row: row['LANES_Text'] + " lanes" if row['LANES'] > 1 else "1 lane" if row['LANES'] == 1 else "", axis=1)

df_ScSegFGDir_baseNoZero_wDirections = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero,df_Directions,on='Dir',how='left')
display(df_ScSegFGDir_baseNoZero_wDirections)

df_ScSegFGDir_baseNoZero_wDirections['FT_Text'] = df_ScSegFGDir_baseNoZero_wDirections['DirDesc'] + " " + df_ScSegFGDir_baseNoZero_wDirections['FUNCGROUP'] + " (" + df_ScSegFGDir_baseNoZero_wDirections['FT_Description'] + ")"
df_ScSegFGDir_baseNoZero_wDirections = df_ScSegFGDir_baseNoZero_wDirections.fillna("")
df_ScSegFGDir_baseNoZero_wDirections['FG_LANES'] = df_ScSegFGDir_baseNoZero_wDirections['FT_Text']  + ": " + df_ScSegFGDir_baseNoZero_wDirections['LANES_Text']
display(df_ScSegFGDir_baseNoZero_wDirections)

df_ScSegFGDir_baseNoZero_collapse = df_ScSegFGDir_baseNoZero_wDirections.groupby(['SEGID','ScenarioCode'])['FG_LANES'].apply('<br/>'.join).reset_index()

df_ScSegFGDir_baseNoZero_collapse_wScenarioName = pd.DataFrame.merge(df_ScSegFGDir_baseNoZero_collapse, df_Scenarios, on='ScenarioCode',how='left')
df_ScSegFGDir_baseNoZero_collapse_wScenarioName['BaseDesc_HTML'] = "<b>" + df_ScSegFGDir_baseNoZero_collapse_wScenarioName['ScenarioName'] + ":</b><br/>" + df_ScSegFGDir_baseNoZero_collapse_wScenarioName['FG_LANES']

df_ScSegFGDir_baseHTML = df_ScSegFGDir_baseNoZero_collapse_wScenarioName[['SEGID','BaseDesc_HTML']]
display(df_ScSegFGDir_baseHTML)


,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,FT_Description,LANES_Text,DirDesc
0,0006_146.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB
1,0006_146.9,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB
2,0006_149.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB
3,0006_149.9,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB
4,0006_150.6,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB
...,...,...,...,...,...,...,...,...,...
7380,WFRC_8213,Arterial,Base19,D2,5.0,1.0,Minor Collector,1 lane,SB/WB
7381,WFRC_8215,Arterial,Base19,D1,4.0,2.0,Major Collector,2 lanes,NB/EB
7382,WFRC_8215,Arterial,Base19,D2,4.0,2.0,Major Collector,2 lanes,SB/WB
7383,WFRC_8218,CD Road,Base19,D1,31.0,1.0,MM: On-ramp,1 lane,NB/EB


,SEGID,FUNCGROUP,ScenarioCode,Dir,FT,LANES,FT_Description,LANES_Text,DirDesc,FT_Text,FG_LANES
0,0006_146.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB,NB/EB Arterial (Principal Arterial),NB/EB Arterial (Principal Arterial): 1 lane
1,0006_146.9,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB,SB/WB Arterial (Principal Arterial),SB/WB Arterial (Principal Arterial): 1 lane
2,0006_149.9,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB,NB/EB Arterial (Principal Arterial),NB/EB Arterial (Principal Arterial): 1 lane
3,0006_149.9,Arterial,Base19,D2,2.0,1.0,Principal Arterial,1 lane,SB/WB,SB/WB Arterial (Principal Arterial),SB/WB Arterial (Principal Arterial): 1 lane
4,0006_150.6,Arterial,Base19,D1,2.0,1.0,Principal Arterial,1 lane,NB/EB,NB/EB Arterial (Principal Arterial),NB/EB Arterial (Principal Arterial): 1 lane
...,...,...,...,...,...,...,...,...,...,...,...
7380,WFRC_8213,Arterial,Base19,D2,5.0,1.0,Minor Collector,1 lane,SB/WB,SB/WB Arterial (Minor Collector),SB/WB Arterial (Minor Collector): 1 lane
7381,WFRC_8215,Arterial,Base19,D1,4.0,2.0,Major Collector,2 lanes,NB/EB,NB/EB Arterial (Major Collector),NB/EB Arterial (Major Collector): 2 lanes
7382,WFRC_8215,Arterial,Base19,D2,4.0,2.0,Major Collector,2 lanes,SB/WB,SB/WB Arterial (Major Collector),SB/WB Arterial (Major Collector): 2 lanes
7383,WFRC_8218,CD Road,Base19,D1,31.0,1.0,MM: On-ramp,1 lane,NB/EB,NB/EB CD Road (MM: On-ramp),NB/EB CD Road (MM: On-ramp): 1 lane


,SEGID,BaseDesc_HTML
0,0006_146.9,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
1,0006_149.9,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
2,0006_150.6,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
3,0006_152.6,<b>2019 Base:</b><br/>NB/EB Arterial (Minor Ar...
4,0006_152.9,<b>2019 Base:</b><br/>NB/EB Arterial (Principa...
...,...,...
3623,WFRC_8208,<b>2019 Base:</b><br/>NB/EB Arterial (Minor Co...
3624,WFRC_8209,<b>2019 Base:</b><br/>NB/EB Arterial (Major Co...
3625,WFRC_8213,<b>2019 Base:</b><br/>NB/EB Arterial (Minor Co...
3626,WFRC_8215,<b>2019 Base:</b><br/>NB/EB Arterial (Major Co...


In [117]:
df_SegmentDetails = pd.DataFrame()
display(df_SegmentDetails)

for (idx, row) in df_SegIDsAll.iterrows():
    
    _segid = row.loc['SEGID']
    print("SEGID: " + _segid)

    #strFinal = "<b>Segment " + _segid + "</b><br/><br/>"
    strFinal = "<br/>"
    
    for (idx, row) in df_Scenarios.iterrows():
        _scenariocode = row.loc['ScenarioCode']
        _scenarioname = row.loc['ScenarioName']
        #print(_scenariocode)
        #print(_scenarioname)
        
        sText = "<b>" + _scenarioname + "</b><br/>"
        
        #list base conditions for base scenario
        if _scenariocode == 'Base19':
            
            #print(_scenarioname)
            
            df_base = df_ScSegFGDir_baseHTML[df_ScSegFGDir_baseHTML['SEGID']==_segid]
            
            if len(df_base.index) > 0:
                strFinal = strFinal + df_base.iloc[0]['BaseDesc_HTML'] + "<br/><br/>"
                #display(df_base.iloc[0]['BaseDesc_HTML'])
            else:
                strFinal = strFinal + "<b>No lanes in " +_scenarioname + "</b><br/><br/>"
            
            #for (idx, row) in df_base.iterrows():
                
                #print(row.loc['BaseDesc_HTML'])
        else:
            
            df_change = df_Changes_HTML[(df_Changes_HTML['SEGID']==_segid) & (df_Changes_HTML['ScenarioCode_x']==_scenariocode)]
            
            if len(df_change.index) > 0:
                #print(_scenarioname)
                strFinal = strFinal + df_change.iloc[0]['Change_HTML'] + "<br/><br/>" 
                #display(df_change.iloc[0]['Change_HTML'])
    
    #display(strFinal)
    df_SegmentDetails = df_SegmentDetails.append({'SEGID'         : _segid,
                                                  'SegDetail_HTML': strFinal
                                                 },ignore_index=True)
    df_SegmentDetails.index.name = 'ID'
    

""


SEGID: 0006_146.9
SEGID: 0006_149.9
SEGID: 0006_150.6
SEGID: 0006_152.6
SEGID: 0006_152.9
SEGID: 0006_155.9
SEGID: 0006_157.3
SEGID: 0006_157.6
SEGID: 0006_158.5
SEGID: 0006_159.6
SEGID: 0006_159.8
SEGID: 0006_160.3
SEGID: 0006_173.4
SEGID: 0006_173.7
SEGID: 0006_174.0
SEGID: 0006_174.4
SEGID: 0006_174.9
SEGID: 0006_175.6
SEGID: 0006_176.1
SEGID: 0006_177.0
SEGID: 0006_177.2
SEGID: 0006_177.5
SEGID: 0006_177.9
SEGID: 0013_000.0
SEGID: 0013_000.6
SEGID: 0013_001.3
SEGID: 0013_001.6
SEGID: 0013_001.8
SEGID: 0013_002.5
SEGID: 0013_002.9
SEGID: 0013_003.3
SEGID: 0013_004.9
SEGID: 0013_005.6
SEGID: 0015_241.6
SEGID: 0015_242.4
SEGID: 0015_244.8
SEGID: 0015_246.9
SEGID: 0015_248.8
SEGID: 0015_250.3
SEGID: 0015_250.9
SEGID: 0015_251.6
SEGID: 0015_252.5
SEGID: 0015_253.6
SEGID: 0015_255.9
SEGID: 0015_257.3
SEGID: 0015_257.6
SEGID: 0015_258.8
SEGID: 0015_260.0
SEGID: 0015_261.8
SEGID: 0015_263.4
SEGID: 0015_264.2
SEGID: 0015_265.6
SEGID: 0015_266.5
SEGID: 0015_267.9
SEGID: 0015_269.0
SEGID: 001

SEGID: 0089_345.9
SEGID: 0089_346.5
SEGID: 0089_347.4
SEGID: 0089_348.0
SEGID: 0089_348.3
SEGID: 0089_348.5
SEGID: 0089_348.8
SEGID: 0089_349.3
SEGID: 0089_349.8
SEGID: 0089_350.1
SEGID: 0089_350.9
SEGID: 0089_351.4
SEGID: 0089_351.8
SEGID: 0089_352.0
SEGID: 0089_352.2
SEGID: 0089_362.5
SEGID: 0089_363.3
SEGID: 0089_363.8
SEGID: 0089_364.3
SEGID: 0089_364.8
SEGID: 0089_365.5
SEGID: 0089_366.3
SEGID: 0089_366.8
SEGID: 0089_367.2
SEGID: 0089_368.0
SEGID: 0089_368.4
SEGID: 0089_369.0
SEGID: 0089_369.8
SEGID: 0089_369.9
SEGID: 0089_370.6
SEGID: 0089_371.0
SEGID: 0089_371.5
SEGID: 0089_372.0
SEGID: 0089_372.2
SEGID: 0089_372.6
SEGID: 0089_372.7
SEGID: 0089_373.6
SEGID: 0089_374.5
SEGID: 0089_375.3
SEGID: 0089_375.7
SEGID: 0089_376.2
SEGID: 0089_376.8
SEGID: 0089_377.3
SEGID: 0089_377.9
SEGID: 0089_378.0
SEGID: 0089_378.3
SEGID: 0089_378.5
SEGID: 0089_378.6
SEGID: 0089_378.8
SEGID: 0089_378.9
SEGID: 0089_379.1
SEGID: 0089_379.2
SEGID: 0089_379.4
SEGID: 0089_379.5
SEGID: 0089_379.8
SEGID: 008

SEGID: 0171_010.0
SEGID: 0171_010.1
SEGID: 0171_010.4
SEGID: 0171_010.6
SEGID: 0171_010.7
SEGID: 0171_011.0
SEGID: 0171_011.3
SEGID: 0171_011.6
SEGID: 0171_011.9
SEGID: 0171_012.2
SEGID: 0171_012.5
SEGID: 0171_012.7
SEGID: 0171_013.1
SEGID: 0171_013.6
SEGID: 0171_014.1
SEGID: 0171_014.6
SEGID: 0171_015.6
SEGID: 0172_000.0
SEGID: 0172_001.0
SEGID: 0172_002.0
SEGID: 0172_003.0
SEGID: 0172_004.0
SEGID: 0172_004.5
SEGID: 0172_005.0
SEGID: 0172_005.5
SEGID: 0172_006.0
SEGID: 0172_007.0
SEGID: 0172_008.0
SEGID: 0172_008.5
SEGID: 0173_000.0
SEGID: 0173_000.7
SEGID: 0173_001.7
SEGID: 0173_002.2
SEGID: 0173_002.8
SEGID: 0173_003.8
SEGID: 0173_004.4
SEGID: 0173_004.7
SEGID: 0173_005.0
SEGID: 0173_005.3
SEGID: 0173_005.8
SEGID: 0173_006.3
SEGID: 0173_006.8
SEGID: 0173_007.3
SEGID: 0173_007.7
SEGID: 0173_008.8
SEGID: 0173_009.2
SEGID: 0173_009.3
SEGID: 0175_000.0
SEGID: 0175_000.1
SEGID: 0175_000.4
SEGID: 0175_000.9
SEGID: 0175_001.4
SEGID: 0175_001.9
SEGID: 0175_002.4
SEGID: 0175_002.9
SEGID: 017

SEGID: 1422_000.7
SEGID: 1422_001.5
SEGID: 1422_002.0
SEGID: 1424_000.0
SEGID: 1424_000.3
SEGID: 1425_000.0
SEGID: 1425_000.6
SEGID: 1427_000.0
SEGID: 1427_000.1
SEGID: 1428_000.0
SEGID: 1428_000.4
SEGID: 1428_001.1
SEGID: 1428_001.7
SEGID: 1428_004.2
SEGID: 1430_000.0
SEGID: 1431_000.5
SEGID: 1431_001.0
SEGID: 1431_002.8
SEGID: 1432_000.0
SEGID: 1435_000.4
SEGID: 1435_000.9
SEGID: 1435_001.4
SEGID: 1435_002.4
SEGID: 1436_000.0
SEGID: 1436_000.3
SEGID: 1438_000.8
SEGID: 1438_000.9
SEGID: 1438_002.0
SEGID: 1438_002.4
SEGID: 1438_003.6
SEGID: 1438_004.6
SEGID: 1438_005.7
SEGID: 1439_000.0
SEGID: 1442_000.0
SEGID: 1442_000.1
SEGID: 1445_001.4
SEGID: 1445_002.9
SEGID: 1445_003.7
SEGID: 1446_000.0
SEGID: 1446_000.8
SEGID: 1446_001.7
SEGID: 1448_000.2
SEGID: 1449_001.3
SEGID: 1450_000.0
SEGID: 1450_000.8
SEGID: 1452_000.0
SEGID: 1452_000.9
SEGID: 1454_000.0
SEGID: 1454_000.6
SEGID: 1456_000.0
SEGID: 1456_000.2
SEGID: 1456_001.0
SEGID: 1456_001.7
SEGID: 1456_002.7
SEGID: 1456_003.4
SEGID: 145

SEGID: 2094_008.0
SEGID: 2094_009.0
SEGID: 2094_010.0
SEGID: 2094_011.0
SEGID: 2094_011.5
SEGID: 2094_012.1
SEGID: 2094_012.6
SEGID: 2094_013.1
SEGID: 2095_001.1
SEGID: 2097_000.0
SEGID: 2098_000.0
SEGID: 2098_000.4
SEGID: 2098_001.4
SEGID: 2101_000.0
SEGID: 2102_000.0
SEGID: 2102_000.4
SEGID: 2102_000.6
SEGID: 2102_001.1
SEGID: 2102_001.4
SEGID: 2102_002.0
SEGID: 2102_002.3
SEGID: 2102_003.0
SEGID: 2104_000.0
SEGID: 2104_000.8
SEGID: 2104_001.2
SEGID: 2104_001.7
SEGID: 2104_002.2
SEGID: 2104_002.7
SEGID: 2105_000.0
SEGID: 2105_000.5
SEGID: 2105_001.2
SEGID: 2105_001.4
SEGID: 2107_001.0
SEGID: 2107_001.5
SEGID: 2107_002.8
SEGID: 2107_003.0
SEGID: 2107_003.5
SEGID: 2107_004.5
SEGID: 2107_004.7
SEGID: 2108_000.0
SEGID: 2108_000.7
SEGID: 2108_001.3
SEGID: 2113_000.0
SEGID: 2113_001.6
SEGID: 2114_000.0
SEGID: 2114_000.1
SEGID: 2114_000.7
SEGID: 2116_000.0
SEGID: 2116_001.0
SEGID: 2116_001.3
SEGID: 2116_001.7
SEGID: 2116_002.0
SEGID: 2116_003.1
SEGID: 2116_003.6
SEGID: 2116_004.5
SEGID: 211

SEGID: 2258_003.6
SEGID: 2258_004.2
SEGID: 2258_004.5
SEGID: 2258_005.0
SEGID: 2258_005.3
SEGID: 2258_005.7
SEGID: 2262_000.0
SEGID: 2262_000.3
SEGID: 2262_000.4
SEGID: 2262_000.6
SEGID: 2262_000.9
SEGID: 2262_001.2
SEGID: 2262_001.5
SEGID: 2262_001.8
SEGID: 2262_002.3
SEGID: 2262_002.4
SEGID: 2262_002.9
SEGID: 2262_003.4
SEGID: 2264_000.0
SEGID: 2264_000.9
SEGID: 2264_001.8
SEGID: 2264_002.6
SEGID: 2264_003.2
SEGID: 2264_003.8
SEGID: 2264_004.3
SEGID: 2264_004.5
SEGID: 2264_004.8
SEGID: 2264_004.9
SEGID: 2264_005.1
SEGID: 2264_005.2
SEGID: 2264_005.4
SEGID: 2264_005.5
SEGID: 2266_000.0
SEGID: 2266_000.6
SEGID: 2266_001.8
SEGID: 2266_002.4
SEGID: 2266_003.3
SEGID: 2266_004.3
SEGID: 2266_005.0
SEGID: 2266_005.2
SEGID: 2266_006.1
SEGID: 2266_006.7
SEGID: 2266_007.2
SEGID: 2269_000.0
SEGID: 2270_000.0
SEGID: 2270_000.3
SEGID: 2270_000.6
SEGID: 2270_000.9
SEGID: 2270_001.2
SEGID: 2270_001.5
SEGID: 2270_001.8
SEGID: 2270_002.1
SEGID: 2270_002.4
SEGID: 2270_002.7
SEGID: 2270_003.0
SEGID: 227

SEGID: 2894_000.9
SEGID: 2894_001.4
SEGID: 2894_001.7
SEGID: 2894_001.9
SEGID: 2895_000.0
SEGID: 2895_000.9
SEGID: 2895_001.3
SEGID: 2895_002.4
SEGID: 2895_002.9
SEGID: 2895_003.5
SEGID: 2895_004.0
SEGID: 2895_004.4
SEGID: 2895_004.8
SEGID: 2896_000.0
SEGID: 2896_000.5
SEGID: 2896_001.3
SEGID: 2896_001.8
SEGID: 2897_000.0
SEGID: 2897_000.4
SEGID: 2897_001.3
SEGID: 2897_001.8
SEGID: 2897_002.3
SEGID: 2897_002.8
SEGID: 2899_000.0
SEGID: 2899_000.2
SEGID: 2899_000.4
SEGID: 2899_000.8
SEGID: 2899_001.2
SEGID: 2899_001.7
SEGID: 2899_002.4
SEGID: 2899_003.5
SEGID: 2899_004.1
SEGID: 2900_000.0
SEGID: 2900_000.5
SEGID: 2900_000.8
SEGID: 2900_001.3
SEGID: 2900_002.2
SEGID: 2900_002.7
SEGID: 2900_003.2
SEGID: 2901_000.0
SEGID: 2901_000.3
SEGID: 2902_000.0
SEGID: 2902_000.3
SEGID: 2902_000.6
SEGID: 2902_001.1
SEGID: 2902_001.6
SEGID: 2902_002.1
SEGID: 2903_000.0
SEGID: 2906_002.8
SEGID: 2907_000.0
SEGID: 2907_001.0
SEGID: 2909_000.0
SEGID: 2911_000.3
SEGID: 2913_000.3
SEGID: 2915_000.0
SEGID: 291

SEGID: 3070_000.0
SEGID: 3071_000.0
SEGID: 3071_000.8
SEGID: 3071_002.1
SEGID: 3080_000.0
SEGID: 3080_000.4
SEGID: 3082_000.0
SEGID: 3084_000.0
SEGID: 3084_000.5
SEGID: 3084_000.9
SEGID: 3084_001.3
SEGID: 3084_001.7
SEGID: 3086_000.0
SEGID: 3086_000.4
SEGID: 3088_000.0
SEGID: 3090_000.0
SEGID: 3306_004.8
SEGID: 3306_005.5
SEGID: 3308_000.0
SEGID: 3308_000.5
SEGID: 3308_001.5
SEGID: 3308_002.0
SEGID: 3308_002.5
SEGID: 3308_002.7
SEGID: 3310_000.0
SEGID: 3310_000.5
SEGID: 3311_000.0
SEGID: 3315_000.0
SEGID: 3315_000.6
SEGID: 3316_000.0
SEGID: 3316_000.6
SEGID: 3316_000.8
SEGID: 3316_001.6
SEGID: 3318_000.0
SEGID: 3320_000.0
SEGID: 3320_000.3
SEGID: 3320_000.8
SEGID: 3320_001.8
SEGID: 3320_002.4
SEGID: 3322_000.0
SEGID: 3328_000.0
SEGID: 3328_000.6
SEGID: 3328_001.0
SEGID: 3328_001.2
SEGID: 3330_000.0
SEGID: 3336_000.0
SEGID: 3340_000.0
SEGID: 3340_000.8
SEGID: 3342_000.0
SEGID: 3344_000.0
SEGID: 3344_001.5
SEGID: 3344_002.2
SEGID: 3346_000.0
SEGID: 3346_000.8
SEGID: 3346_001.0
SEGID: 334

SEGID: MAG_6228
SEGID: MAG_6229
SEGID: MAG_6230
SEGID: MAG_6231
SEGID: MAG_6234
SEGID: MAG_6235
SEGID: MAG_6236
SEGID: MAG_6237
SEGID: MAG_6238
SEGID: MAG_6241
SEGID: MAG_6242
SEGID: MAG_6243
SEGID: MAG_6247
SEGID: MAG_6248
SEGID: MAG_6249
SEGID: MAG_6250
SEGID: MAG_6251
SEGID: WFRC_8000
SEGID: WFRC_8001
SEGID: WFRC_8003
SEGID: WFRC_8004
SEGID: WFRC_8005
SEGID: WFRC_8006
SEGID: WFRC_8007
SEGID: WFRC_8009
SEGID: WFRC_8013
SEGID: WFRC_8014
SEGID: WFRC_8015
SEGID: WFRC_8017
SEGID: WFRC_8018
SEGID: WFRC_8019
SEGID: WFRC_8020
SEGID: WFRC_8021
SEGID: WFRC_8022
SEGID: WFRC_8023
SEGID: WFRC_8024
SEGID: WFRC_8025
SEGID: WFRC_8026
SEGID: WFRC_8027
SEGID: WFRC_8031
SEGID: WFRC_8034
SEGID: WFRC_8035
SEGID: WFRC_8041
SEGID: WFRC_8042
SEGID: WFRC_8043
SEGID: WFRC_8044
SEGID: WFRC_8046
SEGID: WFRC_8047
SEGID: WFRC_8048
SEGID: WFRC_8049
SEGID: WFRC_8050
SEGID: WFRC_8051
SEGID: WFRC_8052
SEGID: WFRC_8053
SEGID: WFRC_8054
SEGID: WFRC_8055
SEGID: WFRC_8056
SEGID: WFRC_8057
SEGID: WFRC_8058
SEGID: WFRC_80

In [118]:
dirResults

'E:\\GitHub\\V-over-C-Calculations\\results'

In [119]:
df_SegmentDetails.to_json(os.path.join(dirResults, r'WebAppData\segmentdetails.json'),orient='records')

In [120]:
len(df_SegmentDetails.index)

3933

# Forecasts Prep

In [125]:
df_Forecasts = pd.read_csv(os.path.join(dirResults, r'0_Forecasts.csv'))

df_Forecasts_wScenarios = pd.DataFrame.merge(df_Forecasts,df_Scenarios,on='ScenarioCode')

df_Forecasts_wScenarios['ForecastAADT_Rounded'] = (df_Forecasts_wScenarios['ForecastAADT'] / 100).astype(int) *100

df_Forecasts_wScenarios['Forecast_Formatted'] = df_Forecasts_wScenarios['ForecastAADT_Rounded'].map('{:,.0f}'.format)

df_Forecasts_wScenarios['Forecast_HTML'] = "<b>" + df_Forecasts_wScenarios['ScenarioName'] + "</b>: " + df_Forecasts_wScenarios['Forecast_Formatted']

df_Forecasts_wScenarios = df_Forecasts_wScenarios.groupby(['SEGID'])['Forecast_HTML'].apply('<br/>'.join).reset_index()

display(df_Forecasts_wScenarios)

df_Forecasts_wScenarios.to_json(os.path.join(dirResults, r'WebAppData\forecastdetails.json'),orient='records')


,SEGID,Forecast_HTML
0,0006_000.0,<b>2019 Base</b>: 0<br/><b>2024 TIP</b>: 0<br/...
1,0006_000.7,<b>2019 Base</b>: 0<br/><b>2024 TIP</b>: 0<br/...
2,0006_016.0,<b>2019 Base</b>: 0<br/><b>2024 TIP</b>: 0<br/...
3,0006_046.0,<b>2019 Base</b>: 0<br/><b>2024 TIP</b>: 0<br/...
4,0006_060.2,<b>2019 Base</b>: 0<br/><b>2024 TIP</b>: 0<br/...
...,...,...
8503,WFRC_8215,"<b>2019 Base</b>: 3,700<br/><b>2024 TIP</b>: 4..."
8504,WFRC_8216,"<b>2019 Base</b>: 0<br/><b>2024 TIP</b>: 12,00..."
8505,WFRC_8217,<b>2019 Base</b>: 0<br/><b>2024 TIP</b>: 0<br/...
8506,WFRC_8218,"<b>2019 Base</b>: 25,500<br/><b>2024 TIP</b>: ..."


# Seg File Export

In [122]:
#for (idx, row) in sdf_Forecasts.iterrows():
#
#    segid = row.loc['SEGID']
#    
#    #print(segid)   
#    
#    df_export = df_VCcmb[(df_VCcmb['SEGID']==segid) & (df_VCcmb['VCGroupCode'].str.contains('DY')==False) & (df_VCcmb['VCGroupCode'].str.contains('FrPM')==False)]
#    df_export = df_export.drop(columns=['SEGID'])
#    
#    df_export_s_fg = df_export.groupby(['ScenarioCode','FUNCGROUP','AREATYPE','FT','LANES','Cap1Hr','DOWFACFC','SsnGrp','ForecastAADT'],as_index=False).agg({'VCGroupCode':[np.size]})
#    df_export_s_fg.columns = ['ScenarioCode','FUNCGROUP','AREATYPE','FT','LANES','Cap1Hr','DOWFACFC','SsnGrp','ForecastAADT','numRec']
#    df_export_s_fg_vc = df_export[['ScenarioCode','FUNCGROUP','VCGroupCode','Dir','SeasonFactor','DOWFactor','PercentTrucksMD','PercentTrucksHV','VolPercentSegDY','PHF','VolMaxHourPercent','ForecastPeakHour','ForecastPeak15MinPCE','VCPeak15MinPCE']]
#    
#    filename_s_fg = os.path.join(dirResults,r"WebAppData\segments\\" + segid + '_s_fg.csv')
#    filename_s_fg_vc = os.path.join(dirResults,r"WebAppData\segments\\" + segid + '_s_fg_vc.csv')
#    display('Exporting ' + segid)
#    df_export_s_fg.to_csv(filename_s_fg,index=False)
#    df_export_s_fg_vc.to_csv(filename_s_fg_vc,index=False)

In [123]:
df_export

NameError: name 'df_export' is not defined

In [ ]:
df_VCcmb_base = df_VCcmb[df_VCcmb['ScenarioCode']=='Base19']
display(df_VCcmb_base)

In [ ]:
df_VCcmb_nonbase = df_VCcmb[df_VCcmb['ScenarioCode']!='Base19']
display(df_VCcmb_nonbase)